In [1]:
import cv2
import numpy as np
import pdb

from Managers import *

# Mog background example

In [2]:
mog = cv2.createBackgroundSubtractorMOG2()

In [3]:
def createMogBackground(image):
    return cv2.cvtColor(mog.apply(image), cv2.COLOR_GRAY2BGR)

In [4]:
cam = CameraManager('Mog Background', cv2.VideoCapture(0), createMogBackground).addCloseCallback()
cam.run()

# KNN background detection with cleanup

In [5]:
bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)

In [6]:
def createKnnBackground(image, l_thresh=244, k_height=3, k_width=3, min_size=1600):
    foreground = bs.apply(image)
    _, thresh = cv2.threshold(foreground, l_thresh, 255, cv2.THRESH_BINARY)
    
    ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_height, k_width))
    dilate = cv2.dilate(thresh, ellipse, iterations=2)

    contours, hier = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if cv2.contourArea(c) > min_size:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
    
    return image

In [7]:
cam = CameraManager('Knn Background', cv2.VideoCapture(0), createKnnBackground).addCloseCallback()
cam.run()

# Mean Shift

Mean shift is like clustering but we find a kernel between pixels (one that includes colour properties in the colour distance as well). A group of pixels will move little between frames but the center of gravity will also move. We can't just track the object by taking every point and applying no discrimination, as it will pickup outlier pixels. 

We want to group together similiar pixels that are close and find a local peak. The peak indicates the cluster that the pixels belong to. The peak can change a bit over time as well, but we are modelling an object where pixels share some property and are reasonably close.

In [2]:
class MousePoints:
    def __init__(self):
        self.p1 = None
        self.p2 = None

        
    def getRect(self):
        if not self.p1 or not self.p2: return

        x1, y1 = self.p1
        x2, y2 = self.p2
        x_min = min(x1, x2)
        x_max = max(x1, x2)
        y_min = min(y1, y2)
        y_max = max(y1, y2)
        return (x_min, y_min, x_max-x_min, y_max-y_min)
    
    
    def __call__(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if self.p1 is None:
                self.p1 = (x, y)
            elif self.p2 is None:
                self.p2 = (x, y)

In [3]:
class HistogramTracker:
    def __init__(self, mousePoints):
        self.mousePoints = mousePoints

        self.hsv_roi = None
        self.term_crit = None
        self.roi_hist = None
        self.track_window = None


    def intializeHSVRoi(self, image):
        x, y, w, h = self.track_window

        self.hsv_roi = image[y : y + h, x : x + w]
        self.hsv_roi = cv2.cvtColor(self.hsv_roi, cv2.COLOR_BGR2HSV)
        masked_roi = cv2.inRange(self.hsv_roi, np.array([100., 30., 32.]), np.array([180., 120., 255.]))

        self.roi_hist = cv2.calcHist([self.hsv_roi], [0], masked_roi, [180], [0,180])
        self.roi_hist = cv2.normalize(self.roi_hist, 0, 255, cv2.NORM_MINMAX)
        self.term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

        
    
    def trackHist(self, image):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        back_proj = cv2.calcBackProject([hsv], [0], self.roi_hist, [0, 180], 1)
    
        _, self.track_window = cv2.meanShift(back_proj, self.track_window, self.term_crit)

        x, y, w, h = self.track_window
        cv2.rectangle(image, (x, y), (x + w, y + h), 255, 2)
        return image


    def __call__(self, image):
        if not self.track_window is None:
            image = self.trackHist(image)
            
            cv2.circle(image, self.mousePoints.p1, 2, (19, 134, 242), 5)
            cv2.circle(image, self.mousePoints.p2, 2, (19, 134, 242), 5)

        elif self.mousePoints.p2:
            self.track_window = self.mousePoints.getRect()
            self.intializeHSVRoi(image)

        elif self.mousePoints.p1:
            cv2.circle(image, self.mousePoints.p1, 2, (19, 134, 242), 5)

        return image

In [4]:
mousePoints = MousePoints() # 'Singleton' kind of class
histTracker = HistogramTracker(mousePoints)

cam = CameraManager('Mean Shift', cv2.VideoCapture(0), histTracker, mouseCallback=mousePoints).addCloseCallback()
cam.writeVideo('D:/Meanshift.avi')
cam.run()